In [1]:
import pandas as pd
import numpy as np
import math
import os
import random
from scipy.spatial import distance

def euclidean_distance(a, b):
    """
    Calcula a distância euclidiana entre dois pontos
    
    Parameters:
    
    a (Tuple)(List): Lista ou tupla com N elementos
    b (Tuple)(List): Lista ou tupla com N elementos

    Returns:
    float: Distância euclidiana entre os pontos
    """
    #return distance.euclidean(a,b)
    return distance.cityblock(a,b)

def merge_on_pontoId(df_1, df_2):
    """
    Faz um merge de dois DataFrame usando o pontoId como referência
    """
    return pd.merge(df_1, df_2, how='inner', on=['idx'])

In [ ]:
def GetFingerprintByIndex(index, CDB_Meds):

    PDs_df = CDB_Meds.loc[(CDB_Meds['idx'] == index)]

    return PDs_df

# Erros de localização médio, mínimo, máximo e desvio padrão

In [ ]:
def calculate_errors(df_real, df_predict):
    """
    Retorna uma lista contendo os erros em metros de todos os pontos no real e na predição
    """
    df_merged = merge_on_pontoId(df_predict, df_real)
    errors = []
    
    for idx, point in df_merged.iterrows():
        distance = GeoUtils.distanceInKm(point.lat, point.lon, point.lat_pred, point.lon_pred)
        errors.append(round(distance * 1000, 2))
        
    return errors

def calculate_theorical_errors():
    df_real = pd.read_csv("./databases/meds_pandas.csv")
    dict_errors = {}
    
    for file, method in zip(FILES,METHODS):
        df_predict = pd.read_csv(file)
        dict_errors[method] = calculate_errors(df_real, df_predict)
    
    return pd.DataFrame(dict_errors)

In [ ]:
def get_nearest_point(test_database, point):
    """
    Encontra o ponto no fingerprint que possui a menor distância euclidiana para o ponto passado como argumento
    """
    min_dist = None
    lat = None
    lon = None
    CDB_Reduced = CDB_ReducedByPDs(point['delay_1'], point['delay_2'], point['delay_3'],
                                   point['delay_12'], point['delay_13'], point['delay_23'], test_database)
    for idx, point_fg in CDB_Reduced.iterrows():
        fg = point_fg[:-8]#-2
        dist = euclidean_distance(point[:-7], fg)#-1
        
        if (min_dist == None) or (min_dist > dist):
            min_dist = dist
            lat = point_fg.lat
            lon = point_fg.lon

    return lat, lon

def calculate_coords_th(fingerprint, test_database):
    """
    Calcula as coordenadas usando um fingerprint e retorna um dataframe contendo
    latitude, longitude e pontoId
    """
    points_dict = {'lat_pred': [], 'lon_pred': [], 'idx': []}
    
    for idx, point in tqdm(fingerprint.iterrows()):
        lat, lon = get_nearest_point(test_database, point)
        points_dict['lat_pred'].append(lat)
        points_dict['lon_pred'].append(lon)
        points_dict['idx'].append(point.idx)
    
    
    return pd.DataFrame(points_dict)
    
def generate_theorical_results(FILES, TEST_DATABASE):
    """
    Calcula as latitudes e longitudes estimadas dos pontos da base de teste usando cada um dos fingerprints
    com as dimensões listadas no vetor DIM
    """
    test_database = pd.read_csv(TEST_DATABASE)
    for method, file in FILES:
        fingerprint = pd.read_csv(file)
        points_df = calculate_coords_th(fingerprint, test_database)
        points_df.to_csv("./FinalResult/Resultados_Metodo_{}.csv".format(method), index=False)